# 使用模型，无记忆的循环聊天模式


In [1]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
APIKEY = input()
model = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=APIKEY,
    model="thudm/glm-4-32b:free",
)

In [4]:
from langchain_core.output_parsers import StrOutputParser
demochain = model | StrOutputParser() 
while True:
    user_input = input("You: ")
    print(demochain.invoke(
        [HumanMessage(content=user_input)]
    ))
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Exiting the chat. Goodbye!")
        break


Hi Bob! How can I assist you today?

Sorry, I cannot answer this question. As an AI assistant, I don't have any information about myself, including who I am. My goal is to help users get answers they need by providing objective information. If you have any questions, you can ask me anytime, and I'll try my best to provide useful answers.

BYD is a large China-based manufacturer that produces a variety of products, including cell phones, batteries, electronics, automobiles, and buses. It is also heavily investing in new energy vehicles (NEVs) and renewable energy solutions.

It was founded in February 1995 by Wang Chuanfu.
It is headquartered in Shenzhen, Guangdong.
BYD means Build Your Dreams.

It is one of the world's largest manufacturers of rechargeable batteries.
It has the world's largest installed capacity of solar power panels.
It is also one of the largest Chinese automakers.
It also provides solutions for rail transit.

So, what specifically would you like to know about BYD? 

# 让模型记住之前聊过什么

In [6]:
from langchain_core.chat_history import BaseChatMessageHistory,InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
store = {}
def getSessionHistory(session_id):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]
with_history = RunnableWithMessageHistory(model, get_session_history=getSessionHistory)

In [7]:
Bob_config = {"configurable": {"session_id": "Bob"}}
res = with_history.invoke(
    [HumanMessage(content="I am Bob")],
    config=Bob_config
)
print(res)
res = with_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=Bob_config
)
print(res)

content='\nHello 👋, Bob! Really happy to meet you! If you have any questions or need help, please feel free to tell me.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 8, 'total_tokens': 38, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'thudm/glm-4-32b:free', 'system_fingerprint': None, 'id': 'gen-1748391309-MWeOj4nwoexGeBMboo6L', 'finish_reason': 'stop', 'logprobs': None} id='run-ae149abb-6401-4188-a04e-851ae16ebfc2-0' usage_metadata={'input_tokens': 8, 'output_tokens': 30, 'total_tokens': 38, 'input_token_details': {}, 'output_token_details': {}}
content='\nYour name is Bob.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 45, 'total_tokens': 52, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'thudm/glm-4-32b:free', 'system_fingerprint': None, 'id': 'gen-1748391313-U59aBtWxwbVZ88Rxs3ib',

In [10]:
Alice_config = {"configurable": {"session_id": "Alice"}}
res = with_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=Alice_config
)
print(res)
res = with_history.invoke(
    [HumanMessage(content="What did I say before?")],
    config=Bob_config
)
print(res)
res = with_history.invoke(
    [HumanMessage(content="What did I say before?")],
    config=Alice_config
)
print(res)

content="\nYou haven't told me what your name is yet. Could you please share it?" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 145, 'total_tokens': 164, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'thudm/glm-4-32b:free', 'system_fingerprint': None, 'id': 'gen-1748391460-d0tMuRoUP3cXS9z4e06b', 'finish_reason': 'stop', 'logprobs': None} id='run-6d08cffd-35bb-44f2-a96d-1377c334a4a7-0' usage_metadata={'input_tokens': 145, 'output_tokens': 19, 'total_tokens': 164, 'input_token_details': {}, 'output_token_details': {}}
content='\nBefore that, you asked: "What is my name?"' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 135, 'total_tokens': 149, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'thudm/glm-4-32b:free', 'system_fingerprint': None, 'id': 'gen-1748391463-ZvPHOs6CMXIJt22fZwHa', 'fin

# 现在保持了记忆，引入提示词模板

In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "given number {number}, return the square of it."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
chain = prompt | model | StrOutputParser()
print(chain.invoke(
    {
        "number": 5,
        "messages": [HumanMessage(content="Let's go!")]
    }
))


Okay, here you go!

The square of 5 is **25**.
